# 章節 12：證據

## 12.5 一個更好的模型

到目前為止我們的分析是基於簡化 SAT 所有的問題的難度都一樣。但現實上，有些問題會比其他簡單，也就是說 Alice 和 Bob 的差異會更小。但模型產生的誤差有多大？ 如果很小，我們可以説第一個簡化模型的版本就已經足夠好了。但如果很大，我們需要更好的模型。

之後的小節會開發一個更好的模型，並且發現模型的誤差是小的。所以如果你對簡單的模型感到滿意，你可以跳過到下一個章節。如果你想看更貼近真實的模型怎麼運作，請繼續閱讀。

- 假設每一個測試者有某個功率來代表他們回答 SAT 問題的能力
- 假設每個問題有某個難度的等級

最後，假設測試者答對的機率和功率與問題難度有關，根據以下函數：
<pre>
     def ProbCorrect(efficacy, difficulty, a=1):
         return 1 / (1 + math.exp(-a * (efficacy - difficulty)))
</pre>

上方函數是簡化版本的曲線用在項目反應理論（Item Response Theory），你可以參考 http://en.wikipedia.org/wiki/Item_response_theory 

功率和問題難度在相同的值域，並且答對問題的機率取決於它們的差。當功率和問題難度相同時，答對的機率為 50%。當功率增加機率會趨近 100%。當功率減少時（或問題難度增加），機率會趨近 0％。

給定功率對不同測試者的分佈，以及問題難度的分佈，我們可以計算原始分數的期望值分佈。

以下分成兩個步驟。

1. 給定一個測試者的功率，我們計算原始分數的分佈。

<pre>
def PmfCorrect(efficacy, difficulties):
    pmf0 = thinkbayes.Pmf([0])
    ps = [ProbCorrect(efficacy, diff) for diff in difficulties]
    pmfs = [BinaryPmf(p) for p in ps]
    dist = sum(pmfs, pmf0)
    return dist
</pre>    

- difficulties 是每一問題的難度
- ps 是一序列的機率
- pmfs 是一序列的兩數值的 Pmf 物件

以下是 BinaryPmf 函數的程式碼：

<pre>
def BinaryPmf(p):
    pmf = thinkbayes.Pmf()
    pmf.Set(1, p)
    pmf.Set(0, 1-p)
    return pmf
</pre>

- dist 是這些 Pmfs 的總和。在 5.4 小節當我們加總 Pmf 物件的結果是一個總和的分佈。為了用 Python 的 sum 函數加總，我們傳入 pmf0 讓 pmf+pmf0 總是一個 Pmf 物件。

如果我們知道測試者個功率，我們可以計算他們的原始分數分佈。一群人有著不同的功率，結果是原始分數的混合分佈。以下是程式碼：

<pre>
# class Exam:
    def MakeRawScoreDist(self, efficacies):
        pmfs = thinkbayes.Pmf()
        for efficacy, prob in efficacies.Items():
            scores = PmfCorrect(efficacy, self.difficulties)
            pmfs.Set(scores, prob)
        mix = thinkbayes.MakeMixture(pmfs)
        return mix
</pre>

傳入 MakeRawScoreDist 的功率是一個 Pmf 物件，代表不同測試者的功率分佈。這邊假設是一個平均值為 0 標準差為 1.5 的高斯分佈。

這個選擇大多是任意的。答對問題的機率根據功率跟問題難度的差異，所以我們可以選擇功率單位然後相應的調整問題難度。

pmfs 是一個 meta-pmf 它包含不同功率對問題答對的分佈 Pmf，並且對應一個 prob 為有此功率答題者的比例。MakeMixutre 計算 meta-pmf 的機率混合體（參考 5.6 小節）。